#Decoding emotional content from physiological responses to visual stimuli

This notebook demonstrates how to run a decoding analysis of physiological data obtained from a subject observing emotional visual stimuli.

To get started, let's clone the course github containing our experimental files

In [ ]:
!git clone https://github.com/StolkArjen/interacting-minds.git
# to remove the folder, use: !rm -rf interacting-minds
# to clear all outputs, go to Edit > Clear all outputs, followed by Runtime > Restart

Add the code folder to the path

In [ ]:
import os, sys
sys.path.append(os.getcwd() + '/interacting-minds/code')

Read data from a subject into the workspace

In [ ]:
from read_iworx import read_iworx

data, event = read_iworx(os.getcwd() + '/interacting-minds/data/EMG_FA22/IME')

print(event.value)
print(event.sample)
print(data.label)
print(len(data.trial))
print(len(data.trial[0]))
print(len(data.trial[0][0]))

Data has the following nested fields: .trial .time .label

Event has the following nested fields: .type .sample .value

Let's sort our stimuli per emotion type, either positive (+) or negative (-), and get their sample indices

In [ ]:
posidx = [i for i, e in enumerate(event.value) if e[0] == '+']
negidx = [i for i, e in enumerate(event.value) if e[0] == '-']

possmp = [event.sample[p] for p in posidx]
negsmp = [event.sample[p] for p in negidx]

print(possmp)
print(negsmp)

Now let's extract data samples from around the presentation of each stimulus based on the above sample indices

In [ ]:
import numpy as np

pre  = 1 # in seconds
post = 1
fs   = 1000 # sample frequency (samples per second)

posdat = []
for t in range(len(data.time)): # loop through trials
  for e in range(len(posidx)): # loop through events
      if possmp[e]-pre > data.time[t][0] and possmp[e]+post < data.time[t][-1]:
          idx = data.time[t].index(possmp[e])
          posdat.append(data.trial[t][:,idx-(pre*fs):idx+(post*fs)])

negdat = []
for t in range(len(data.time)): # loop through trials
  for e in range(len(negidx)): # loop through events
      if negsmp[e]-pre > data.time[t][0] and negsmp[e]+post < data.time[t][-1]:
          idx = data.time[t].index(negsmp[e])
          negdat.append(data.trial[t][:,idx-(pre*fs):idx+(post*fs)])

Let's take a look at the averages

In [ ]:
import matplotlib.pyplot as plt

time = range(-pre*fs, post*fs, 1)
posavg = np.nanmean(posdat, axis=0)
negavg = np.nanmean(negdat, axis=0)

fig, axs = plt.subplots(1,5, figsize=(15,5))
for p in range(len(posavg)):
  axs[p].plot(time, posavg[p,:], 'b')
  axs[p].plot(time, negavg[p,:], 'r')
  axs[p].title.set_text(data.label[p])

Here's where we perform the actual decoding. Let's try to decode positive and negative emotional cues from the physiological responses. In brief, we combine the positive and negative trials into one dataset. Next, we create a labels vector with 0 and 1s corresponding to the positive and negative trials. Finally, we assess statistically whether we can fit a hyperplane that differentiates between the two classes (positive, negative). We use cross-validation to this effect.

In [ ]:
import numpy as np
from sklearn import svm
from sklearn.model_selection import cross_val_score

# create the data matrix and labels
X = np.concatenate((posdat, negdat), axis=0) # trials x measures x samples
y = np.concatenate((np.zeros(len(posdat)), np.ones(len(negdat))), axis=0)
print(X.shape, y.shape)

# use only HR and SC for decoding
hridx = data.label.index('Heart Rate')
scidx = data.label.index('Skin Conductance')
X = X[:,[hridx, scidx],:]

# specify the classifier
clf = svm.SVC(kernel='linear', random_state=42) # support vector classification

# loop across multiple segments for moment-by-moment decoding
winlen   = 25 # window size in samples
segments = np.array_split(X, X.shape[2]/winlen, axis=2)
scores   = []
for s in segments:
  seg = np.reshape((s), (s.shape[0], s.shape[1]*s.shape[2])) # collapse 2nd and 3rd dims (the features)
  scores.append(cross_val_score(clf, seg, y, cv=5, scoring='accuracy')) # 5-fold cross-val
  print("%0.2f accuracy with a standard deviation of %0.2f" % (scores[-1].mean(), scores[-1].std()))

# plot mean accuracy over time
avgacc  = np.nanmean(scores, axis=1)
segtime = np.linspace(time[0], time[-1], num=len(segments))
plt.plot(segtime, avgacc)

How to continue from here? One way would be to test the mean decoding accuracy across participants against the chance-level accuracy (e.g., using a one-sample Student t-test) to verify whether the physiological responses include information about the label (e.g., positive/negative emotional cue). Alternatively, the resulting decoding accuracy can be compared with a permutation distribution of accuracy scores derived from randomly partioning labels or indices between the two datasets (positive, negative) myriad times. The latter can be achieved as follows (note that for the point of demonstration n_permutations is set to too small a number for generating a proper distribution and corresponding p-value).

In [ ]:
from sklearn.model_selection import permutation_test_score

# create and loop across consecutive segments for moment-to-moment decoding
winlen   = 25 # window size in samples
scores   = []
perms    = []
pvalues  = []
for count, s in enumerate(segments):
  print("classifying segment " + str(count+1) + "/" + str(len(segments)))
  seg = np.reshape((s), (s.shape[0], s.shape[1]*s.shape[2])) # collapse 2nd and 3rd dims (the features)
  score, perm, pvalue = permutation_test_score(clf, seg, y, scoring="accuracy", cv=5, n_permutations=20)
  scores.append(score) # scores on original data
  perms.append(perm)   # scores on permutated data
  pvalues.append(pvalue)

In [ ]:
# plot mean score on original and permuted data
fig, ax = plt.subplots()
ax.hist(np.mean(perms, axis=1), bins=20, density=True)
ax.axvline(np.mean(scores), ls="--", color="r")
score_label = f"Score on original\ndata: {np.mean(scores):.2f}\n(p-value: {np.mean(pvalues):.3f})"
ax.text(0.7, 10, score_label, fontsize=12)
ax.set_xlabel("Accuracy score")
_ = ax.set_ylabel("Probability")

# plot accuracy and p-value over time
fig, ax = plt.subplots()
segtime = np.linspace(time[0], time[-1], num=len(segments))
plt.plot(segtime, scores)
plt.plot(segtime, pvalues)
ax.legend(['Accuracy score', 'P-value'])

For further reading on support vector machines and cross-validation, have a look at https://scikit-learn.org/stable/modules/svm.html and https://scikit-learn.org/stable/modules/cross_validation.html